In [ ]:
from pathlib import Path

import altair as alt
import pandas as pd
import numpy as np

import qiime2
from q2_composition import DataLoafPackageDirFmt

In [ ]:
def plot_differentials(category, df, output_dir,
                        q_threshold=1.0, 
                        lfc_abs_threshold=0, 
                        sort_by='lfc', 
                        feature_ids=None,
                        figsize=(25, 25)):

    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True) 
    fig_fp = output_dir / Path(f'{category}-ancombc-barplot.html')
    
    if feature_ids is not None:
        # only display the feature-ids in the 
        # feature_ids list
        raise NotImplementedError
        df = df[df['feature-id'] in feature_ids]
    
    df = df[df['q_val'] <= q_threshold]
    df = df[np.abs(df['lfc']) >= lfc_abs_threshold]
    df = df.sort_values(sort_by)
    df['q_val_text'] = [f'{q:.2e}' for q in df['q_val']]
    
    bars = alt.Chart(df).mark_bar().encode(
        x='lfc',
        y='feature-id'
    )

    text = bars.mark_text(
        align='left',
        baseline='middle',
        dx=3  # Nudges text to right so it doesn't appear on top of the bar
    ).encode(
        text='q_val_text'
    )

    chart = (bars + text).properties(height=900)
    chart.save(fig_fp)

In [ ]:
loaf = qiime2.Artifact.load('./ancombc-body_habitat.qza')

slices = loaf.view(DataLoafPackageDirFmt)
data = {}
for e in slices.data_slices.iter_views(pd.DataFrame):
    data[str(e[0]).replace('_slice.csv','')] = e[1]
columns = [e for e in data['lfc'].columns if e not in '(Intercept)']

figure_data = []
for category in columns:
    if category == 'id':
        continue
    df = pd.concat([data['lfc']['id'], 
                    data['lfc'][category], 
                    data['se'][category], 
                    data['q_val'][category]],
                   keys=['feature-id', 'lfc', 'se', 'q_val'], axis=1)
    figure_data.append((category, df))
    plot_differentials(category, df, './da-plots/', q_threshold=0.001)